In [20]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [21]:
df = pd.read_csv(r"C:\Users\thinp\Desktop\Graduate School\Data\jeju_card\201901-202003.csv")
df_april = pd.read_csv(r"C:\Users\thinp\Desktop\Graduate School\Data\jeju_card\202004.csv")
sub = pd.read_csv(r"C:\Users\thinp\Desktop\Graduate School\Data\jeju_card\submission.csv")

In [22]:
df = df[df.REG_YYMM <= 202001]

In [23]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [24]:
# 날짜 처리

df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

In [25]:
# 데이터 정제
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [26]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [8]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [9]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(n_jobs=-1, random_state=0)

In [10]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [2]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [11]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,0,0,0,1,1,1,2020,2,359616.0
1,0,0,0,1,1,2,2020,2,420384.0
2,0,0,0,1,1,3,2020,2,383701.0
3,0,0,0,1,1,4,2020,2,2325287.0
4,0,0,0,1,1,5,2020,2,2325287.0
...,...,...,...,...,...,...,...,...,...
829425,16,30,14,0,2,1,2020,2,129950.0
829426,16,30,14,0,2,2,2020,2,140701.0
829427,16,30,14,0,2,3,2020,2,140701.0
829428,16,30,14,0,2,4,2020,2,140701.0


In [12]:
train_num_2 = train_num.drop(['CSTMR_CNT', 'CNT'], axis=1)

In [13]:
train_num_2 = pd.concat([train_num_2, temp])

In [14]:
# feature, target 설정
train_num_2 = train_num_2.sample(frac=1, random_state=0)
train_features = train_num_2.drop(['AMT'], axis=1)
train_target = np.log1p(train_num_2['AMT'])

In [15]:
# 훈련
model2 = RandomForestRegressor(n_jobs=-1, random_state=0)
model2.fit(train_features, train_target)

RandomForestRegressor(n_jobs=-1, random_state=0)

In [16]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [3]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [17]:
# 예측
pred = model2.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,0,0,0,1,1,1,2020,3,510896.0
1,0,0,0,1,1,2,2020,3,447703.0
2,0,0,0,1,1,3,2020,3,383931.0
3,0,0,0,1,1,4,2020,3,2325287.0
4,0,0,0,1,1,5,2020,3,2366974.0
...,...,...,...,...,...,...,...,...,...
829425,16,30,14,0,2,1,2020,3,133032.0
829426,16,30,14,0,2,2,2020,3,138097.0
829427,16,30,14,0,2,3,2020,3,140743.0
829428,16,30,14,0,2,4,2020,3,140785.0


In [18]:
train_num_3 = pd.concat([train_num_2, temp])

In [19]:
train_num_3.to_csv('./train_num_3.csv')

In [3]:
train_ex = pd.read_csv('./train_num_3.csv')
train_ex

TypeError: parser_f() got an unexpected keyword argument 'index'

In [6]:
train_num_3 = train_ex.drop(['Unnamed: 0'], axis=1)

In [10]:
train_num_3.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
484517,7,31,3,2,1,1,2020,1,177900.0
1516695,9,13,3,1,2,5,2020,2,85365.0
1982162,4,12,13,3,2,3,2020,3,77717.0
290352,5,33,3,5,1,2,2020,2,1018709.0
1461439,4,14,3,1,1,3,2020,2,391439.0


In [7]:
# feature, target 설정
train_num_3 = train_num_3.sample(frac=1, random_state=0)
train_features = train_num_3.drop(['AMT'], axis=1)
train_target = np.log1p(train_num_3['AMT'])

In [8]:
# 훈련
model3 = RandomForestRegressor(n_jobs=-1, random_state=0)
model3.fit(train_features, train_target)

RandomForestRegressor(n_jobs=-1, random_state=0)

In [11]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = train_num_3['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = train_num_3['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = train_num_3['HOM_SIDO_NM'].unique()
AGEs          = train_num_3['AGE'].unique()
SEX_CTGO_CDs  = train_num_3['SEX_CTGO_CD'].unique()
FLCs          = train_num_3['FLC'].unique()
years         = [2020]
months        = [4]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [13]:
# 예측
pred = model3.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,7,31,3,2,1,1,2020,4,197782.0
1,7,31,3,2,1,5,2020,4,200615.0
2,7,31,3,2,1,3,2020,4,276999.0
3,7,31,3,2,1,2,2020,4,423280.0
4,7,31,3,2,1,4,2020,4,230630.0
...,...,...,...,...,...,...,...,...,...
829425,6,8,4,0,2,1,2020,4,46194.0
829426,6,8,4,0,2,5,2020,4,46132.0
829427,6,8,4,0,2,3,2020,4,46192.0
829428,6,8,4,0,2,2,2020,4,46194.0


In [17]:
temp['REG_YYMM'] = (temp['year']*100) + temp['month']
temp.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT,REG_YYMM
0,7,31,3,2,1,1,2020,4,197782.0,202004
1,7,31,3,2,1,5,2020,4,200615.0,202004
2,7,31,3,2,1,3,2020,4,276999.0,202004
3,7,31,3,2,1,2,2020,4,423280.0,202004
4,7,31,3,2,1,4,2020,4,230630.0,202004


In [18]:
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [27]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [28]:
# 제출 파일 만들기
submission = pd.read_csv(r"C:\Users\thinp\Desktop\Graduate School\Data\jeju_card\submission.csv",index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp,
                              left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],
                              right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],
                              how='left')
submission.index.name = 'id'
submission.to_csv('./submission/submission_recurrent_RF1.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,6.853307e+08
1,202004,강원,골프장 운영업,1.633877e+09
2,202004,강원,과실 및 채소 소매업,2.382076e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.141898e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.083992e+08


In [29]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [30]:
general = pd.read_csv(r'C:\Users\thinp\python\Kaggle\jeju_card\submission\submission_baseline_RF1.csv', index_col=0)
general

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,4.959178e+08
1,202004,강원,골프장 운영업,6.951267e+09
2,202004,강원,과실 및 채소 소매업,1.818448e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.011042e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.810176e+07
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.833437e+09
1390,202007,충북,한식 음식점업,2.425573e+10
1391,202007,충북,호텔업,1.980399e+08


In [31]:
god = pd.read_csv(r'C:\Users\thinp\python\Kaggle\jeju_card\submission\submission_final.csv', index_col=0)
god

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.437028e+09
1390,202007,충북,한식 음식점업,1.948352e+10
1391,202007,충북,호텔업,1.472099e+07


In [36]:
jo1 = pd.read_csv(r'C:\Users\thinp\python\Kaggle\jeju_card\submission\submission_12month_RF1.csv', index_col=0)
jo2 = pd.read_csv(r'C:\Users\thinp\python\Kaggle\jeju_card\submission\submission_12month_RF2.csv', index_col=0)
jo1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,5.672182e+08
1,202004,강원,골프장 운영업,7.352978e+09
2,202004,강원,과실 및 채소 소매업,2.162199e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.116197e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.059692e+08
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.785763e+09
1390,202007,충북,한식 음식점업,3.123945e+10
1391,202007,충북,호텔업,3.002983e+08


In [37]:
jo1 = jo1[jo1.REG_YYMM ==202004]
jo2 = jo2[jo2.REG_YYMM ==202004]


In [32]:
recurrent = submission[submission.REG_YYMM == 202004]
recurrent

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,6.853307e+08
1,202004,강원,골프장 운영업,1.633877e+09
2,202004,강원,과실 및 채소 소매업,2.382076e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.141898e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.083992e+08
...,...,...,...,...
692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,9.021533e+08
693,202004,충북,한식 음식점업,1.160729e+10
694,202004,충북,호텔업,2.737500e+08


In [33]:
general = general[general.REG_YYMM == 202004]
god = god[god.REG_YYMM == 202004]

In [40]:
top = pd.read_csv(r'C:\Users\thinp\python\Kaggle\jeju_card\submission\submission_features_RF1.csv', index_col=0)
top = top[top.REG_YYMM == 202004]

In [34]:
rmsle(god['AMT'], general['AMT'])

# 제주 가중치 전 7.3255827310
# 제주 가중치 후 7.0952800022	
# 약 0.23 차이가 납니다.

7.325582731001321

In [35]:
rmsle(god['AMT'], recurrent['AMT'])

7.462976901303662

In [38]:
rmsle(god['AMT'], jo1['AMT'])

7.5308433680497515

In [39]:
rmsle(god['AMT'], jo2['AMT'])

7.32826832619835

In [41]:
rmsle(god['AMT'], top['AMT'])

# 제주 가중치 전 1.7128441400
# 제주 가중치 후 1.6341797485
# 약 0.08 차이가 납니다...

1.7128441400013001